In [2]:
import pandas as pd
from tqdm import tqdm
from mistralai import Mistral
import time
import re

api_key = None


In [3]:
chunks_df = pd.read_csv('merged_chunks.csv')
questions = pd.read_csv('questions.csv')
wikipedia_summaries = pd.read_csv('wikipedia_summaries.csv', header=None)

In [28]:
def generate_prompt_reflexion_with_sources_and_answer(body, possible_answer_a, possible_answer_b, possible_answer_c, possible_answer_d, possible_answer_e, chunks, wikipedia_summaries, reflexion_allowed):
    prompt = "Vous êtes un expert en médecine, santé et biologie. Pour la question suivante, évaluez attentivement chaque réponse.\n"
    if reflexion_allowed :
        prompt += "Souvent, une seule réponse est correcte. Réfléchissez bien et écrivez votre raisonnement brièvement avant de donner votre réponse finale.\n"
    else:
        prompt += "Souvent, une seule réponse est correcte. Réfléchissez bien mais n'écrivez pas votre raisonnement avant de donner votre réponse finale.\n"
    prompt += "À la fin de votre raisonnement, fournissez la ou les réponse(s) au format '[A]', ou '[A,B,C]', par exemple.\n"
    prompt += "Votre réponse finale doit être une liste de lettres sans crochets et sans espaces, triées par ordre alphabétique.\n\n"
    prompt += f"Question : {body}\n"
    prompt += f"A : {possible_answer_a}\n"
    prompt += f"B : {possible_answer_b}\n"
    prompt += f"C : {possible_answer_c}\n"
    prompt += f"D : {possible_answer_d}\n"
    prompt += f"E : {possible_answer_e}\n\n"

    prompt += "Pour chaque réponse possible, vous disposez des informations supplémentaires suivantes :\n"
    prompt += f"A : A wikipedia article related : {wikipedia_summaries[0][:500]}"
    prompt += f"A : Some more informations : {chunks[0][:400]}\n"
    prompt += f"B : A wikipedia article related : {wikipedia_summaries[1][:500]}"
    prompt += f"B : Some more informations : {chunks[1][:400]}\n"
    prompt += f"C : A wikipedia article related : {wikipedia_summaries[2][:500]}"
    prompt += f"C : Some more informations : {chunks[2][:400]}\n"
    prompt += f"D : A wikipedia article related : {wikipedia_summaries[3][:500]}"
    prompt += f"D : Some more informations : {chunks[3][:400]}\n"
    prompt += f"E : A wikipedia article related : {wikipedia_summaries[4][:500]}"
    prompt += f"E : Some more informations : {chunks[4][:400]}\n\n"

    prompt += "Si aucune information supplémentaire n'est fournie pour une certaine réponse, cela peut signifier que cette réponse n'est pas liée à la question, augmentant la probabilité qu'elle soit fausse.\n\n"

    prompt += "Je vous rappelle que votre réponse doit être parmi :\n"
    prompt += f"A : {possible_answer_a}\n"
    prompt += f"B : {possible_answer_b}\n"
    prompt += f"C : {possible_answer_c}\n"
    prompt += f"D : {possible_answer_d}\n"
    prompt += f"E : {possible_answer_e}\n\n"

    if reflexion_allowed:
        prompt += "Vous êtes un expert en médecine, santé et biologie : ne manquez pas votre réponse. Souvent, une seule réponse est correcte. Ecrivez un raisonnement plutôt bref, puis fournissez votre réponse finale sous la forme [A] ou [B,D,E] par exemple :\n"
    else:
        prompt += "Vous êtes un expert en médecine, santé et biologie : ne manquez pas votre réponse. Souvent, une seule réponse est correcte. N'écrivez pas de raisonnement, fournissez votre réponse finale sous la forme 'A' ou 'B,D,E' (séparez bien par des virgules les différentes lettres et mettez les entre des guillemets'') par exemple ici :\n"

    return prompt

In [5]:
def answer_prompt_with_Mistral(prompt, temperature=0.):
    client = Mistral(api_key=api_key)
    chat_response = client.chat.complete(
                model="mistral-large-latest",
                messages=[
                    {
                        "role": "user",
                        "content": prompt
                    },
                ],
                temperature=temperature
            )
    return chat_response.choices[0].message.content

In [26]:
def get_submission_file(chunks, wikipedia_summaries, q, reflexion_allowed):

    def inserer_virgules(answer):
        nouvelle_chaine = ""
        for i in range(len(answer)):
            nouvelle_chaine += answer[i]
            if i < len(answer) - 1 and answer[i].isalpha() and answer[i+1].isalpha():
                nouvelle_chaine += ","
        return nouvelle_chaine

    submission = []
    for i in tqdm(range(len(chunks_df))):
    # for i in tqdm(range(10)):
        chunks_i = chunks.iloc[i].tolist()
        wikipedia_summaries_i = wikipedia_summaries.iloc[i]
        prompt = generate_prompt_reflexion_with_sources_and_answer(q.iloc[i]['question'], q.iloc[i]['answer_A'],
                                                                   q.iloc[i]['answer_B'], q.iloc[i]['answer_C'],
                                                                    q.iloc[i]['answer_D'], q.iloc[i]['answer_E'],
                                                                    chunks=chunks_i, wikipedia_summaries=wikipedia_summaries_i,
                                                                    reflexion_allowed=reflexion_allowed)
        while True:
            try:
                answer = answer_prompt_with_Mistral(prompt, temperature=0.)
                break
            except:
                time.sleep(1)
        submission.append(answer)
    submission_df = pd.DataFrame(submission)
    submission_df.columns = ['Answer']
    submission_df.index.name = 'id'

    for i in range(len(submission_df)):
        submission_df['Answer'][i] = inserer_virgules(submission_df['Answer'][i]) 
    submission_df.to_csv('submission.csv')
    return submission    

In [29]:
sub = get_submission_file(chunks_df, wikipedia_summaries, questions, reflexion_allowed=True)

100%|██████████| 103/103 [27:26<00:00, 15.98s/it]
/var/folders/l5/v7xs7rjj7471_h1k0y_1ygxw0000gp/T/ipykernel_3075/425641457.py:33: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  submission_df['Answer'][i] = inserer_virgules(submission_df['Ans